In [11]:
import json
from pii_detection.gen_data_utils import rewrite_essay
from tqdm import tqdm
from pathlib import Path

GEN_DIR = Path.cwd().parent / "data" / "generated"
GEN_DIR.mkdir(parents=True, exist_ok=True)

with open("../data/train_shard.json") as f:
    data = json.load(f)


for essay in tqdm(data):
    padded_doc_id = str(essay["document"]).zfill(6)
    dst_json = GEN_DIR / f"{padded_doc_id}.json"
    
    if not dst_json.exists():
        new_essay = rewrite_essay(essay, debug=True)

        with dst_json.open("w") as f:
            json.dump(essay, f, indent=2)


  0%|          | 1/4768 [00:34<46:12:18, 34.89s/it]

[DEBUG] OUTPUT: Challenge and Selection

I work as a system engineer in a major French corporation, but I also have an entrepreneurial side as an application developer with my brother. Our business model as entrepreneurs is to develop mobile applications for smartphones. We have been inspired by two books, "Fail Fast or Win Big" by Bernard Schroeder and "Art of Innovation" by Tom Kelley. Our challenge can be summarized as "Big win or Fail fast": we aim to be efficient and quickly validate or reject our ideas through lean prototyping. However, due to limited resources and time, we face the challenge of deciding which ideas to pursue.

To overcome this challenge, we have implemented a two-step learning launch approach for each idea: Step 1 is to determine if the application addresses a real customer need, and Step 2 is to verify if the application effectively meets this need in terms of human-machine interface, functionality, and technical difficulty.

To pass Step 1, I try to put myself

  0%|          | 2/4768 [01:05<42:43:32, 32.27s/it]

[DEBUG] OUTPUT: Challenge:
As a technology program manager for my organization, my role is to oversee the development and implementation of new platforms to replace outdated ones. The goal is to increase efficiency in business processes and reduce technical debt. We recently developed a new platform and had high hopes for its quick adoption and the eventual decommissioning of the old platform. We assumed that the new platform's enhanced functionality and user-friendly features would make it easily embraced by users. However, in reality, we faced resistance and adoption rates were low, despite conducting comprehensive training sessions both on-site and via webcasts. This forced us to make several adjustments to our rollout plans, causing frustration among management and technology teams who had to support both platforms simultaneously. The general perception was that the training material was poorly designed and the training sessions needed to be more practical and engaging.

Selection:

  0%|          | 3/4768 [01:25<35:22:55, 26.73s/it]

...

[DEBUG] OUTPUT: Design Thinking Tool: Mind Mapping

Challenge:

I reside in Buenos Aires, the capital city of Argentina and the most populous city in the country. During the COVID-19 pandemic, the government imposed a strict 60-day quarantine, and we are currently in mandatory preventive isolation with limitations on public transportation and business operations. At the bank where I work, we were faced with the challenge of transitioning from remote work to returning to the office.

To address this challenge, my team and I needed to understand the diverse realities of our 1,500 employees and their perspectives on the pandemic. We wanted to know their thoughts, feelings, and needs to ensure that the transition to the office would be tailored to their requirements and beneficial for the bank. In my opinion, the best tool for identifying these needs was mind mapping. It would provide a comprehensive view of employee needs, allowing us to determine whether we could meet them immediately or

  0%|          | 6/4768 [02:47<36:55:17, 27.91s/it]

[DEBUG] OUTPUT: Design Thinking: Final Summative Assessment

1. Challenge: Describe the challenge, including all relevant information.

In order to address the unequal access to quality higher education around the world, the challenge was to create a platform that would provide education to the masses. The goal was to bridge the gap between the education received in prestigious universities like Ivy League schools and the education received in less renowned colleges in developing countries. The challenge was to develop a platform that would value the time of both students and teachers while expanding access, lowering costs, and increasing flexibility.

2. Selection: In your own words, briefly describe the tool you selected (e.g., what it is and why you selected it for your challenge -- including any appropriate video lecture references).

The tool selected for this challenge was storytelling. By using storytelling, the aim was to inspire individuals to improve their circumstances and a

  0%|          | 7/4768 [03:02<31:24:57, 23.75s/it]

[DEBUG] OUTPUT: Reflective Visualization

The first point of the course emphasized the importance of deductive thinking, which involves moving from the general to the specific. This approach proved to be the most applicable, as it helped us develop a clearer understanding of the concepts we wanted to convey. Additionally, by relying on visualization, we were able to create a more visually appealing representation of our ideas through drawings, enabling a greater level of human comprehension.

In my personal experience, I have found this tool to be incredibly useful in teaching economics. Economics, being a branch of social sciences, is often difficult to visualize, but with the help of visualization techniques, my students are able to grasp the concepts more easily.

Visualization is a underutilized tool, particularly in the field of economics education. While we do have graphs that assist in visually representing data, the topics and thought processes covered in this course have allow